# Supervised Regression with XGBoost Algorithm: Predict AirBnB Listing Prices

## Problem Statement: 

Using the XGBoost algorithm we will predict the AirBnB listing prices based on available features such as **host information** like response rate, location, etc., and **listing information** such as number of bedrooms, number of bathrooms, etc. 

## 0. Setup Google Colab Environment 

In [2]:
!python3 -m pip install xgboost numpy pandas scikit-learn plotnine --quiet 
!python3 -m pip install seaborn pyarrow optuna plotly --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 11.9 MB/s eta 0:00:00


## 1. Import Libraries and Modules

In [3]:
# Import of built-in libraries.
import os 
import warnings

# Import of third-party libraries.
import IPython
import numpy as np 
import pandas as pd  
import matplotlib as mpl 
import seaborn as sns 
import xgboost as xgb

In [4]:
# Print versions of imported libraries.
print('ipython --version: %s' %(IPython.__version__))
print('numpy --version: %s' %(np.__version__))
print('pandas --version: %s' %(pd.__version__))
print('matplotlib --version: %s' %(mpl.__version__))
print('seaborn --version: %s' %(sns.__version__))
print('xgboost --version: %s' %(xgb.__version__))

ipython --version: 7.9.0
numpy --version: 1.21.6
pandas --version: 1.3.5
matplotlib --version: 3.2.2
seaborn --version: 0.11.2
xgboost --version: 0.90


In [5]:
# Ignore warnings
warnings.filterwarnings(action='ignore')

In [6]:
# Import required modules from libraries

from IPython.display import display 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from matplotlib import pyplot as plt 

In [7]:
pd.set_option('display.max_columns', 1000)

## 2. Load Data Set as Pandas DataFrame

In [16]:
data_loc = 'https://raw.githubusercontent.com/rs2pydev/reg_xgb_airbnb/main/dataset/listings.csv'
listings_orig = pd.read_csv(filepath_or_buffer=data_loc, header='infer')

## 3.1 Analytical EDA

In [17]:
# Shape of DataFrame.
print('Shape of DataFrame: ', listings_orig.shape)

Shape of DataFrame:  (39881, 75)


In [21]:
# Display first few rows of DataFrame.
display(listings_orig.head())

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2539,https://www.airbnb.com/rooms/2539,20220907064715,2022-09-07,city scrape,Clean & quiet apt home by the park,Renovated apt home in elevator building.<br />...,Close to Prospect Park and Historic Ditmas Park,https://a0.muscache.com/pictures/3949d073-a02e...,2787,https://www.airbnb.com/users/show/2787,John,2008-09-07,"New York, NY",Educated professional living in Brooklyn. I l...,within an hour,100%,90%,f,https://a0.muscache.com/im/pictures/user/86745...,https://a0.muscache.com/im/pictures/user/86745...,Gravesend,9.0,12.0,"['email', 'phone']",t,t,"Brooklyn , New York, United States",Kensington,Brooklyn,40.64529,-73.97238,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,"[""First aid kit"", ""Microwave"", ""Stove"", ""Coffe...",$299.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,21,51,81,356,2022-09-07,9,0,0,2015-12-04,2018-10-19,4.89,4.88,5.00,5.00,5.00,4.75,4.88,NaN,f,9,1,6,2,0.11
1,2595,https://www.airbnb.com/rooms/2595,20220907064715,2022-09-07,city scrape,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, NY",A New Yorker since (Phone number hidden by Air...,within a day,75%,23%,f,https://a0.muscache.com/im/pictures/user/50fc5...,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,6.0,9.0,"['email', 'phone', 'work_email']",t,t,"New York, United States",Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,"[""Stove"", ""Coffee maker"", ""Long term stays all...",$175.00,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,0,0,5,280,2022-09-07,49,1,0,2009-11-21,2022-06-21,4.68,4.73,4.63,4.77,4.80,4.81,4.40,NaN,f,3,3,0,0,0.31
2,5121,https://www.airbnb.com/rooms/5121,20220907064715,2022-09-07,city scrape,BlissArtsSpace!,One room available for rent in a 2 bedroom apt...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,Garon,2009-02-03,"New York, NY","I am an artist(painter, filmmaker) and curato...",within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/72a61...,https://a0.muscache.com/im/pictures/user/72a61...,Bedford-Stuyvesant,2.0,2.0,"['email', 'phone']",t,t,NaN,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,Private room,2,NaN,NaN,1.0,1.0,"[""Heating"", ""Kitchen"", ""Air conditioning"", ""Wi...",$60.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,5,30,60,335,2022-09-07,50,0,0,2009-05-28,2019-12-02,4.52,4.22,4.09,4.91,4.91,4.47,4.52,NaN,f,2,0,2,0,0.31
3,45910,https://www.airb

In [23]:
# DataFrame Metadata Information 
listings_orig.iloc[:, 0:20].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39881 entries, 0 to 39880
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     39881 non-null  int64 
 1   listing_url            39881 non-null  object
 2   scrape_id              39881 non-null  int64 
 3   last_scraped           39881 non-null  object
 4   source                 39881 non-null  object
 5   name                   39868 non-null  object
 6   description            39036 non-null  object
 7   neighborhood_overview  23466 non-null  object
 8   picture_url            39881 non-null  object
 9   host_id                39881 non-null  int64 
 10  host_url               39881 non-null  object
 11  host_name              39831 non-null  object
 12  host_since             39831 non-null  object
 13  host_location          32998 non-null  object
 14  host_about             22509 non-null  object
 15  host_response_time 

In [15]:
# Print all column names.
col_names = listings.columns.to_list()
print('Column names: ', col_names, sep='\n', end='\n\n')

Column names: , ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability

- There are 75 columns in total: 74 features and 1 target variable, namely, `price`. 

- For further analysis, we will first create a Pandas DataFrame corresponding to the features and a Pandas Series corresponding to the target. 

- Next, as a first step of the analysis, we will focus on discarding a few features based on their values (data content).  

In [ ]:
X = listings.drop(columns=['price'])

In [ ]:
listings.head(3)

## 3.2 Visual EDA